# Machine, Data & Learning

## Project | Genetic Algorithm

Team 30 | Tribrid
-----------
* Nitin Chandak (2019101024)
* Ayush Sharma (2019101004)

In [99]:
# Usefull Imports
import client as server
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import json
%matplotlib inline

### Initial Overfit Vector
```
[
    0.0, 
    -1.45799022e-12, 
    -2.28980078e-13,  
    4.62010753e-11, 
    -1.75214813e-10, 
    -1.83669770e-15,  
    8.52944060e-16,  
    2.29423303e-05, 
    -2.04721003e-06, 
    -1.59792834e-08,  
    9.98214034e-10
]
```

In [100]:
# Usefull Global Constants & lambda functions
TEAM_ID = 'colthAUIKUTfdh4qWrnHBhJzkyEm8kt4qIue1BKtyvLItfp8Po'
# INITIAL_OVERFIT_VECTOR = [
#     0.0, 
#     -1.45799022e-12, 
#     -2.28980078e-13,  
#     4.62010753e-11, 
#     -1.75214813e-10, 
#     -1.83669770e-15,  
#     8.52944060e-16,  
#     2.29423303e-05, 
#     -2.04721003e-06, 
#     -1.59792834e-08,  
#     9.98214034e-10
# ]
# INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.38544571e-12, -2.39314288e-13,  4.62010753e-11,
#  -1.86809773e-10, -1.88180339e-15,  8.87303182e-16,  2.17511080e-05,
#  -2.02847485e-06, -1.59792834e-08,  9.94492532e-10,]
# INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.34157007e-12, -2.43502736e-13,  4.18030722e-11,
#  -1.70243951e-10, -1.88180339e-15,  9.47209422e-16,  2.20840660e-05,
#  -2.11341874e-06, -1.59792834e-08,  9.01093380e-10,]
INITIAL_OVERFIT_VECTOR=[ 0.00000000e+00, -1.34157007e-12, -2.39500664e-13,  3.93964654e-11,
 -1.76372543e-10, -1.79528508e-15,  9.74197339e-16,  2.00010867e-05,
 -2.21274832e-06, -1.62774610e-08,  8.39523570e-10 ]
POPULATION_SIZE = 8
TOTAL_GENERATION = 10
MIN_GENE_VAL = -10
MAX_GENE_VAL = 10
MUTATION_PROBABILITY = 0.3
MUTATION_FACTOR = lambda : random.uniform(0.9, 1.1)
MATING_POOL_SIZE = lambda : random.randint(3,5)

In [101]:
def mutate_initial_individual(individual):
    '''
    This function will mutate every gene in an individual which is
    a vector of 11-D or array of 11 size with probability
    0.8 & leave other gene as it is.
    '''
    ret_individual = []
    for igene in range(len(individual)):
        ret_individual.append(individual[igene])
        if random.randint(1, 10) <= (10*0.8):
            final_gene = INITIAL_OVERFIT_VECTOR[igene]*MUTATION_FACTOR()
            if final_gene<MIN_GENE_VAL:
                ret_individual[igene] = MIN_GENE_VAL
            elif final_gene>MAX_GENE_VAL:
                ret_individual[igene] = MAX_GENE_VAL
            else:
                ret_individual[igene] = final_gene
    return ret_individual
            

def create_initial_population():
    '''
    Initializes the population. Gene is I's genenitial_Overfit_Vector's gene by default
    but replaced with MUTATION_FACTOR*Initial_Overfit_Vector
    with probability 0.8.
    Constraint on Gene is:
    MIN_GENE_VAL <= Gene <= MAX_GENE_VAL
    '''
    population = []
    for i in range(POPULATION_SIZE):
        individual = np.array(INITIAL_OVERFIT_VECTOR)
#         individual[individual != 0 ] = 0
        individual = np.array(mutate_initial_individual(individual))
        population.append(individual)
    return population

In [102]:
def population_fitness(population):
    curr_fitness=[]
    for individual in population:
        curr_fitness.append(individual_fitness(individual))
    return curr_fitness


def individual_fitness(individual):
    [train_error, validation_error]= server.get_errors(TEAM_ID,individual.tolist())
    print(train_error, validation_error)
    return  (1 / ((train_error + validation_error) + 2 * abs(train_error - validation_error)))

In [106]:
def sort_y_wrt_x(cx_list,cy_list):
    for i in range(POPULATION_SIZE):
        for j in range(i,POPULATION_SIZE):
            if cx_list[i] > cx_list[j]:
                cx_list[i],cx_list[j] = cx_list[j],cx_list[i]
                cy_list[i],cy_list[j] = cy_list[j],cy_list[i]
    return cx_list,cy_list

In [107]:
def cross_over(mating_population):
    mated_population = []
    index = len(mating_population)-1
    while index>0:
        I1 = mating_population[index]
        I2 = mating_population[index-1]
        crossover_point = random.randint(1, POPULATION_SIZE-1)
        C1 = list(I1[:crossover_point]) + list(I2[crossover_point:])
        C2 = list(I2[:crossover_point]) + list(I1[crossover_point:])
        mated_population.append(C1)
        mated_population.append(C2)
        index-=2
    return mated_population

In [108]:
def mutate_mated_population(mated_population):
    mutated_mated_population = []
    for individual in mated_population:
        temp_individual = []
        for igene in range(len(individual)):
            if random.randint(1, 10) <= (10*MUTATION_PROBABILITY):
                new_gene = individual[igene]*MUTATION_FACTOR()
                if new_gene<MIN_GENE_VAL:
                    temp_individual.append(MIN_GENE_VAL)
                elif new_gene>MAX_GENE_VAL:
                    temp_individual.append(MAX_GENE_VAL) 
                else:
                    temp_individual.append(new_gene)
            else:
                temp_individual.append(individual[igene])
        mutated_mated_population.append(temp_individual)
    return mutated_mated_population

In [109]:
def next_generation(mutated_mated_population,modified_population):
    next_gen_pop = mutated_mated_population[:]
    modified_pop = modified_population[::-1]
    remain_individual = POPULATION_SIZE-len(next_gen_pop)
    for i in range(remain_individual):
        next_gen_pop.append(modified_pop[i])
    return next_gen_pop

In [110]:
def reproduction_function(curr_fitness,curr_population):
    modified_fitness,modified_population = sort_y_wrt_x(list(curr_fitness),curr_population)
    mating_population = modified_population[(POPULATION_SIZE-MATING_POOL_SIZE()):]
    mated_population = cross_over(mating_population)
    mutated_mated_population = mutate_mated_population(mated_population)
    next_gen_population = next_generation(mutated_mated_population,modified_population)
    return next_gen_population

In [114]:
def get_best_vector(curr_fitness, curr_population):
    optimum_fitness_index = -1
    optimum_fitness = -1
    for ifitness in range(len(curr_fitness)):
        if curr_fitness[ifitness]>optimum_fitness:
            optimum_fitness_index = ifitness
    return curr_population[optimum_fitness_index],curr_fitness[optimum_fitness_index]

In [116]:
def run_algo():
    print('hi')
    curr_population = np.array(create_initial_population())
#     curr_fitness =  np.random.randint(1,101,POPULATION_SIZE)
    curr_fitness = np.array(population_fitness(curr_population))
    best_vector,best_vector_fitness = get_best_vector(curr_fitness, curr_population)
    curr_gen = 2
    while curr_gen<=TOTAL_GENERATION:
        print("cuurr_gen => ",curr_gen)
        print("cuurr_population => ",curr_population)
        curr_population = np.array(reproduction_function(curr_fitness,curr_population))
        curr_fitness = np.array(population_fitness(curr_population))
#         curr_fitness = np.random.randint(1,101,POPULATION_SIZE)
        temp_best_vector,temp_best_vector_fitness = get_best_vector(curr_fitness, curr_population)
        if temp_best_vector_fitness>best_vector_fitness:
            best_vector_fitness = temp_best_vector_fitness
            best_vector = temp_best_vector
        curr_gen+=1
    return best_vector,best_vector_fitness

In [117]:
print('hi')
best_vector,best_vector_fitness = run_algo()

hi
hi
54878370437809.625 97175826031677.95
64399615506990.4 115727606559247.75
27240510057222.098 48130991052979.055
45452884872118.39 81271454880065.98
17137527570318.395 28496163714272.375
4764857230660.222 7036029506024.53
21383313583837.51 36813857429584.58
44040388992790.23 78014891001457.61
cuurr_gen =>  2
cuurr_population =>  [[ 0.00000000e+00 -1.28041738e-12 -2.24221534e-13  3.65542920e-11
  -1.71257910e-10 -1.93437078e-15  1.00531753e-15  2.19477226e-05
  -2.42705664e-06 -1.62774610e-08  8.28861517e-10]
 [ 0.00000000e+00 -1.21806158e-12 -2.34718804e-13  3.93964654e-11
  -1.91315882e-10 -1.79528508e-15  9.74197339e-16  2.00010867e-05
  -2.40577665e-06 -1.62774610e-08  8.03876413e-10]
 [ 0.00000000e+00 -1.45131510e-12 -2.24604687e-13  3.93773282e-11
  -1.76372543e-10 -1.63301231e-15  1.06046289e-15  1.92139202e-05
  -2.25232387e-06 -1.75601997e-08  8.99111787e-10]
 [ 0.00000000e+00 -1.34157007e-12 -2.23025950e-13  3.56764137e-11
  -1.66828048e-10 -1.62593928e-15  1.02660232e-15 

57578778665881.79 102811920325091.1
44040388992790.23 78014891001457.61
50959391929859.67 90861138973175.64
28212424215665.27 49202508609575.75
44040388992790.23 78014891001457.61
44040388992790.23 78014891001457.61
44040388992790.23 78014891001457.61
44040388992790.23 78014891001457.61
cuurr_gen =>  7
cuurr_population =>  [[ 0.00000000e+00 -1.31056068e-12 -2.30559458e-13  3.69815298e-11
  -1.61216412e-10 -1.78797945e-15  9.34077798e-16  2.00010867e-05
  -2.21274832e-06 -1.77727407e-08  7.72447217e-10]
 [ 0.00000000e+00 -1.31056068e-12 -2.09834343e-13  3.69815298e-11
  -1.61216412e-10 -1.78797945e-15  9.34077798e-16  2.00010867e-05
  -2.21274832e-06 -1.77727407e-08  8.17164801e-10]
 [ 0.00000000e+00 -1.31056068e-12 -2.52323086e-13  3.69815298e-11
  -1.77329146e-10 -1.78797945e-15  9.19333293e-16  2.00010867e-05
  -2.21274832e-06 -1.86241543e-08  8.10923538e-10]
 [ 0.00000000e+00 -1.31056068e-12 -2.30559458e-13  3.54899188e-11
  -1.61216412e-10 -1.75032375e-15  8.87103879e-16  2.0001086

In [118]:
print(best_vector,best_vector_fitness)
print(server.submit(TEAM_ID,best_vector.tolist()))
individual_fitness(best_vector)

[ 0.00000000e+00 -1.31056068e-12 -2.30559458e-13  3.69815298e-11
 -1.61216412e-10 -1.78797945e-15  9.34077798e-16  2.00010867e-05
 -2.21274832e-06 -1.77727407e-08  8.17164801e-10] 5.2630392267315416e-15
successfully submitted
44040388992790.23 78014891001457.61


5.2630392267315416e-15

In [120]:
IOF=[ 0.00000000e+00, -1.34157007e-12, -2.43502736e-13,  4.18030722e-11,
 -1.70243951e-10, -1.88180339e-15,  9.47209422e-16,  2.20840660e-05,
 -2.11341874e-06, -1.59792834e-08,  9.01093380e-10]
print(server.submit(TEAM_ID,IOF))

successfully submitted
